<a href="https://colab.research.google.com/github/kazuki-de/git_reserch_code/blob/master/IMSAT_256pix_mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0rc0
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D, Activation , Softmax, MaxPool2D,UpSampling2D, Input, BatchNormalization, add, AveragePooling2D
from tensorflow.keras import Model,Sequential
import datetime
#import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image

from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow_hub as hub
 
from tensorflow.keras import layers


     |████████████████████████████████| 380.5MB 80kB/s 
     |████████████████████████████████| 501kB 23.9MB/s 
     |████████████████████████████████| 4.3MB 40.3MB/s 


In [3]:
print(tf.__version__)

2.0.0-rc0


In [2]:
from google.colab import drive
drive.mount("/content/drive")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2" #@param {type:"string"}


In [0]:
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(224,224,3))
feature_extractor_layer.trainable = True


In [18]:
model1 = tf.keras.Sequential([
  feature_extractor_layer,
  layers.Dense(128,activation="relu"),
  layers.Dense(3)
])
 
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense_4 (Dense)              (None, 128)               163968    
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 387       
Total params: 2,422,339
Trainable params: 2,388,227
Non-trainable params: 34,112
_________________________________________________________________


In [0]:
def rescell(data, filters, kernel_size, option=False):
    strides=(1,1)
    if option:
        strides=(2,2)
    x=Conv2D(filters=filters,kernel_size=kernel_size,strides=strides,padding="same")(data)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    
    data=Conv2D(filters=int(x.shape[3]), kernel_size=(1,1), strides=strides, padding="same")(data)
    x=Conv2D(filters=filters,kernel_size=kernel_size,strides=(1,1),padding="same")(x)
    x=BatchNormalization()(x)
    x=add([x,data])
    x=Activation('relu')(x)
    return x

def ResNet(img_rows, img_cols, img_channels):
    input=Input(shape=(img_rows,img_cols,img_channels))
    x=Conv2D(16,(2,2),activation="relu")(input)
    x=Conv2D(32,(7,7), padding="same",activation="relu")(x)
    x=MaxPool2D(pool_size=(2,2))(x)




    x=rescell(x,64,(3,3))
    x=rescell(x,64,(3,3))
    x=rescell(x,64,(3,3))

    x=rescell(x,128,(3,3),True)

    x=rescell(x,128,(3,3))
    x=rescell(x,128,(3,3))
    x=rescell(x,128,(3,3))

    x=rescell(x,256,(3,3),True)

    x=rescell(x,256,(3,3))
    x=rescell(x,256,(3,3))
    x=rescell(x,256,(3,3))
    x=rescell(x,256,(3,3))
    x=rescell(x,256,(3,3))

    x=rescell(x,512,(3,3),True)

    x=rescell(x,512,(3,3))
    x=rescell(x,512,(3,3))

    x=AveragePooling2D(pool_size=(int(x.shape[1]),int(x.shape[2])),strides=(2,2))(x)

    x=Flatten()(x)
    x=Dense(units=3,kernel_initializer="he_normal")(x)
    model=Model(inputs=input,outputs=[x])
    return model
model1 = ResNet(512,512,3)

In [0]:
input_tensor = Input(shape=(img_width, img_height, 3))
ResNet50 = ResNet50(include_top=False, weights='imagenet',input_tensor=input_tensor)
 入力画像は224×224のRGB画像なので，これをinput_tensorとして指定しています．今回

In [0]:
model1.summary()

In [0]:
def kl_divergence(q_logi, p_logi):
    q_logit = Activation("softmax")(q_logi)
    p_logit = Activation("softmax")(p_logi)
    qlogq = tf.reduce_sum(q_logit * tf.math.log(q_logit), axis=1)
    qlogp = tf.reduce_sum(q_logit * tf.math.log(p_logit), axis=1)
    return qlogq - qlogp
def get_unit_vector(v):
    return  v / (tf.sqrt(tf.reduce_sum(v ** 2, axis=None,keepdims=True) + 1e-16))

In [0]:
def compute_entropy(p):
    return - tf.reduce_sum(p*tf.math.log(p+1e-16),axis = 1)

def compute_marginal_entropy(p_batch):
    return  compute_entropy(tf.reduce_mean(p_batch))

In [0]:
#損失関数
loss_object = tf.keras.losses.CategoricalCrossentropy()
#最適化手法
optimizer = tf.keras.optimizers.Adam()

#評価関数
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_loss1 = tf.keras.metrics.Mean(name='train_loss')
train_loss2 = tf.keras.metrics.Mean(name='train_loss')


train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')

test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [0]:
#ミニバッチ学習==1epocでn枚の画像からx枚ずつ取り出しn/x回学習する(重みを更新していく)手法
#バッチ学習 == 1epocでn枚の画像から1枚ずつ取り出しn回学習する（重みを更新していく）手法
#オンライン学習 ==1epocでn枚の画像からn枚全てを用い学習する(重みを更新していく)手法　#メモリ消費大
@tf.function
def train_step(image,label,batch_size):
    with tf.GradientTape() as tape:
        p = Activation('softmax')(model1(image))
        p_ave = tf.reduce_mean(p,axis=0)
        loss_eq2 = -tf.reduce_sum(p_ave * tf.math.log(p_ave + 1e-16))
        loss_eq1 = tf.reduce_mean(compute_entropy(p))
        loss_eq = loss_eq1 - 4* loss_eq2
        #with tf.variable_scope(tf.get_variable_scope(),reuse=True):
        #logit = model1(image)
        
        #---------------------------------------------------------------------------------
        plain_softmax = model1(image)
        
        noplain_softmax = Activation("softmax")(plain_softmax)

        #適当な単位ベクトルにxi=10を掛けた数:pertubationの作成
        perturbation = 10 * get_unit_vector(tf.random.normal(shape=tf.shape(image), dtype=tf.float32))
        #print(perturbation.numpy())
        for i in range(1):
            softmax_accommodating_perturbation = model1(image+perturbation)
            softmax_accommodating_perturbation_af = Activation("softmax")(softmax_accommodating_perturbation)
            # ノイズを足した配列とノーマルな配列がCNNで出力された結果のKL距離を求める
            dist = kl_divergence(noplain_softmax,softmax_accommodating_perturbation_af)
            #cross_entropy_accommodating_perturbation = -tf.reduce_sum(plain_softmax * tf.math.log(clipped(softmax_accommodating_perturbation))) * weight
            adversarial_direction = tf.gradients(dist, [perturbation])[0]
            print(adversarial_direction)
            pertubation = tf.stop_gradient(adversarial_direction)
        #print("ok")    

        pertubation = 1 * get_unit_vector(adversarial_direction)
        corrent_softmax = Activation('softmax')(model1(image)) 
        vat_softmax = Activation('softmax')(model1(image + perturbation))
        VAT_loss = kl_divergence(corrent_softmax,vat_softmax)
        #---------------------------------------------------------------------------------------------
        
        total_loss = VAT_loss +0.2 * loss_eq
        #hy_ = compute_entropy(p)
        #p_ave = tf.reduce_mean(p,axis=0)
        #hy_x = -tf.reduce_sum(p_ave*tf.math.log(p_ave+1e-16))
        #print(hy)
        #hy_x = tf.reduce_sum(compute_entropy(p))/batch_size
        #print(hy_x)
        #Rsut = -tf.reduce_sum(VAT_KL(image,model1))/batch_size
        #print(Rsut)
        #loss = Rsut - 0.2*((4)*hy-hy_x)
    gradients = tape.gradient(total_loss, model1.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model1.trainable_variables))
    
    train_loss(total_loss)
    #train_loss1(Rsut)
    #train_loss2(0.2*((4.0)*hy-hy_x))
    #train_loss(loss)
    #train_loss
    
    
        

In [19]:
%pip install munkres
from munkres import Munkres, print_matrix

In [0]:
def compute_accuracy(y_pred, y_t,tot_cl=10):
    # compute the accuracy using Hungarian algorithm
    m = Munkres()
    mat = np.zeros((tot_cl, tot_cl))
    for i in range(tot_cl):
        for j in range(tot_cl):
            mat[i][j] = np.sum(np.logical_and(y_pred == i, y_t == j))
    indexes = m.compute(-mat)

    corresp = []
    for i in range(tot_cl):
        corresp.append(indexes[i][1])

    pred_corresp = [corresp[predicte] for predicted in y_pred]
    acc = np.sum(pred_corresp == y_t) / float(len(y_t))
    return acc

In [0]:
path = "/content/drive/My Drive/2048pix_tissue_choice/sumpling_20.csv"
def generater_train():
    file_path1 = pd.read_csv("/content/drive/My Drive/224_choice/224_3000.csv")
    file_path1 = file_path1[0:2992]
    
    # LabelEncode(classをint型に変換)するためのdict\n",
    classes =["PI","PP","TRU"]
    classes = {v: i for i, v in enumerate(sorted(classes))}
    #file_path1 = pd.read_csv(file_path)
    length = len(file_path1)
    for i in range(length):
        path = file_path1.loc[i]["base_path"]
        path1 = "/content/drive/My Drive/224_choice/"+path
        with Image.open(path1) as f:
            
                tmp_image = np.asarray(f, dtype=np.float32)/255
        label = file_path1.loc[i]["type"]
        tmp_label = tf.keras.utils.to_categorical(classes[label],len(classes))
        #print(tmp_labels, np.mean(tmp_images, axis=(1,2,3)))
        yield tmp_image, tmp_label

In [0]:
#mnist用
EPOCHS = 100
#model = MyModel()
batchsize = 10
dataset_t = tf.data.Dataset.from_generator(generater_train, output_types=(tf.float32, tf.int32)).batch(16)
#dataset_v = tf.data.Dataset.from_generator(generater_val, output_types=(tf.float32, tf.int32)).batch(10)
for epoch in range(EPOCHS):
    for batch, (x_t,x_l) in enumerate(dataset_t):
        train_step(x_t,x_l,16)
        #print( train_loss.result())
        if batch%100 ==0:
            print(batch)
    model1.save_weights('/content/drive/My Drive/research_/weight_autoencoder_2048_0827/IMSAT_512.h5')
    print( train_loss.result())        

Tensor("gradients/sequential_2_2/keras_layer/StatefulPartitionedCall_grad/PartitionedCall:0", shape=(16, 224, 224, 3), dtype=float32)
Tensor("gradients/sequential_2_2/keras_layer/StatefulPartitionedCall_grad/PartitionedCall:0", shape=(16, 224, 224, 3), dtype=float32)
0
100


In [16]:
file_path1 = pd.read_csv("/content/drive/My Drive/224_choice/224_3000.csv")
file_path1 = file_path1[0:2992]
len(file_path1)

2992

In [0]:
model1.save_weights('/content/drive/My Drive/research_/weight_autoencoder_2048_0827/IMSAT_cifer10.h5')

In [0]:
model1.load_weights('/content/drive/My Drive/research_/weight_autoencoder_2048_0827/IMSAT_mnist.h5')

In [0]:
def VAT_KL(input_tensor, be_softmax, xi=10, epsilon=1.0, weight=1.0, num_approximation=1, clip_value_min=1e-16, dtype=tf.float64):
    
    #配列内の数値の最大値が第二変数になるように置き換える
    #clipped = lambda x: tf.maximum(x, clip_value_min)
    #axis_without_batch_size は (1,2,3)となる
    #axis_without_batch_size = tuple(range(1,len(input_tensor.get_shape())))
    
    #if len(axis_without_batch_size) == 1:
    #    axis_without_batch_size = axis_without_batch_size[0]
        
    #normalized = lambda x: x / clipped(tf.norm(x, axis=None))
    #normalized = lambda x: x / clipped(tf.norm(x, axis=None,keepdims=True))
    #normalized = lambda x: x / tf.norm(x, axis=None,keepdims=True)
    #plain_softmax = network(input_tensor)
    
    #noplain_softmax = Activation("softmax")(plain_softmax)
    
    #適当な単位ベクトルにxi=10を掛けた数:pertubationの作成
    perturbation = xi * get_unit_vector(tf.random.normal(shape=tf.shape(input_tensor), dtype=dtype))
    #print(perturbation.numpy())
    
    for i in range(num_approximation):
        
        softmax_accommodating_perturbation = model1(input_tensor+perturbation)
        
        #softmax_accommodating_perturbation_af = Activation("softmax")(softmax_accommodating_perturbation)
        # ノイズを足した配列とノーマルな配列がCNNで出力された結果のKL距離を求める
        dist = tf.reduce_mean(kl_divergence(be_softmax,softmax_accommodating_perturbation))
        #cross_entropy_accommodating_perturbation = -tf.reduce_sum(plain_softmax * tf.math.log(clipped(softmax_accommodating_perturbation))) * weight
        adversarial_direction = tf.gradients(dist, [perturbation])[0]
        pertubation = tf.stop_gradient(adversarial_direction)
    #print("ok")    
    
    pertubation = epsilon * get_unit_vector(pertubation)
    
    #corrent_softmax = model1(input_tensor) 
    vat_softmax = model1(input_tensor + perturbation)
    loss =  tf.reduce_mean(kl_divergence(be_softmax,vat_softmax))
    return  loss

In [0]:
y = x_test[0:20]
x = Activation('softmax')(model1(y))

In [0]:
file_path1 = pd.read_csv("/content/drive/My Drive/512_choice/512_3000.csv")

In [48]:
file_path1.head()

,type,ID,file_path_before,sd,file_path,base_path
0,PP,TCGA-78-7154-01A-01-TS1,/mnt/Doc/512/PP/TCGA-78-7154-01A-01-TS1/5_8_8....,62.079063,/mnt/Doc/512_choice/PP/TCGA-78-7154-01A-01-TS1...,PP/TCGA-78-7154-01A-01-TS1/5_8_8.jpeg
1,PI,TCGA-73-4658-01A-01-TS1,/mnt/Doc/512/PI/TCGA-73-4658-01A-01-TS1/6_5_14...,59.211271,/mnt/Doc/512_choice/PI/TCGA-73-4658-01A-01-TS1...,PI/TCGA-73-4658-01A-01-TS1/6_5_14.jpeg
2,TRU,TCGA-55-6543-01A-01-BS1,/mnt/Doc/512/TRU/TCGA-55-6543-01A-01-BS1/4_3_1...,61.448240,/mnt/Doc/512_choice/TRU/TCGA-55-6543-01A-01-BS...,TRU/TCGA-55-6543-01A-01-BS1/4_3_1.jpeg
3,PP,TCGA-78-7156-01A-01-TS1,/mnt/Doc/512/PP/TCGA-78-7156-01A-01-TS1/4_4_11...,58.248361,/mnt/Doc/512_choice/PP/TCGA-78-7156-01A-01-TS1...,PP/TCGA-78-7156-01A-01-TS1/4_4_11.jpeg
4,PP,TCGA-50-5932-01A-01-BS1,/mnt/Doc/512/PP/TCGA-50-5932-01A-01-BS1/23_19_...,56.939832,/mnt/Doc/512_choice/PP/TCGA-50-5932-01A-01-BS1...,PP/TCGA-50-5932-01A-01-BS1/23_19_13.jpeg


In [0]:
#file_path1 = pd.read_csv("/content/drive/My Drive/2048pix_tissue_choice/val_1000.csv")
file_path2 =file_path1[400:410]
tmp_images = np.zeros((len(file_path2), 512, 512, 3))
count = 400
for i in range(10):
    
    path = file_path1.loc[i]["base_path"]
    path1 = "/content/drive/My Drive/512_choice/"+path
    count+=1
    with Image.open(path1) as f:
        tmp_images[i] = np.asarray(f, dtype=np.float64)/255

In [0]:
pred =Activation('softmax')(model1(tmp_images[1:5]))

In [67]:
pred

<tf.Tensor: id=95354, shape=(4, 3), dtype=float32, numpy=
array([[0.333333  , 0.33333346, 0.33333358],
       [0.333333  , 0.33333346, 0.33333358],
       [0.33333296, 0.33333343, 0.33333355],
       [0.333333  , 0.33333346, 0.33333358]], dtype=float32)>

In [0]:
xi = []
for i in range(10000):
    print(np.argmax(x[i]))
    xi.append(np.argmax(x[i]))

In [0]:
xi.count(6)

In [0]:

for i in range(10):
    x+=x[1][i]

In [0]:
x[1][0]+x[1][1]+x[1][2]+x[1][3]+x[1][4]+x[1][5]+x[1][6]+x[1][7]+x[1][8]+x[1][9]

In [0]:
np.argmax(x[3])